!pip install --upgrade "ibm-watson>=7.0.1"

# Description

The goal of these notebook is to sucessfully download the actions and upload them witht he new Custom
extension ids in case we modified them for some reason. 



In [2]:
import pprint
from dotenv import load_dotenv
import os
load_dotenv()

True

# Assistant setup 

In [3]:
URL = 'https://api.us-east.assistant.watson.cloud.ibm.com/'
API_KEY = os.getenv(key='API_KEY')

# put here the name of the assistant to replace extensions ids 
#ASSISTANT_NAME = 'Experiments_Ask_Proc_Main'

CHANGE_LOG_DESCRIPTION = '''
----- Process Description -----
Copy all from Ask_Proc_Main to DEV_TEST

----- Changes -----



'''

ASSISTANT_FROM = 'Ask_Proc_Main'
ASSISTANT_TO = 'PREPROD_PROD_AskProc'

BACKUP_FOLDER = 'assistant_backups/'

ONLY_EPM_API_CHANGE = True

MODIFY = [
    {
        'id_from': "6c319007-41f4-4cce-8368-7a6d38905888", #  epm ask_proc_main
        'id_to': "8acb5354-2e54-401a-805a-1e9927167311" # epm_api_v21 ask_proc_main
        },
        {
        'id_from': "0710d532e446af11a39ab1c8d3d4b5281b94cfd91550eb8f2cf6d9ff669b3290", #   epm ask_proc_main
        'id_to': "0710d532e446af11a39ab1c8d3d4b5281b94cfd91550eb8f2cf6d9ff669b3290" # epm_api_v21 ask_proc_main
        },
        {
        'id_from': "2a9c63f0-5e9d-44a8-9c6e-92a5684030be", #  watsonx ask_proc_main
        'id_to': "299017c6-1add-44c3-a5dd-8f7cb2eecb8a" # watsonx ask_proc_main
        },
        {
        'id_from': "cc50493e455de183d661b95360196ddb468a02051add6dcaf4a6479b34900be5", #   watsonx ask_proc_main
        'id_to': "cc50493e455de183d661b95360196ddb468a02051add6dcaf4a6479b34900be5" # watsonx ask_proc_main
        }
]

In [4]:
# Auhentification 
import json
from ibm_watson import AssistantV2
from ibm_cloud_sdk_core.authenticators import IAMAuthenticator

authenticator = IAMAuthenticator(API_KEY)
assistant = AssistantV2(
    version='2021-06-14',
    authenticator = authenticator
)

assistant.set_service_url(URL)



# Create folder for backup for data and logs 

In [5]:
# get current datetime in string
import datetime as dt
date_name = dt.datetime.now().strftime("%Y-%m-%d_%H_%M_%S")
folder_name = date_name
BACKUP_FOLDER = BACKUP_FOLDER+folder_name
date_name

'2024-02-20_11_45_25'

In [6]:
os.makedirs(BACKUP_FOLDER)

In [7]:
BACKUP_FOLDER

'assistant_backups/2024-02-20_11_45_25'

In [8]:
logs_path = BACKUP_FOLDER+'/CHANGE_LOG_DESCRIPTION.txt'
with open(logs_path, mode='w') as f:
    f.write(CHANGE_LOG_DESCRIPTION)
    f.close()

# Changes setup 
In our case we only have 2 custom expensions and only one is the most commun to have changes so that is why I have to flows 
One when I only want to change the ids for EPM api And other when I want to change epm and watsonx.ai ids 


# Getting the assistant ID using the assistant name

In [9]:
assistant_list = assistant.list_assistants()

In [10]:
# find your assistant by name 


for bot in assistant_list.get_result().get('assistants'):
    if bot.get('name') == ASSISTANT_FROM:
        ASSISTANT_FROM_ID = bot.get('assistant_id')
        pprint.pprint(bot)
    if bot.get('name') == ASSISTANT_TO:
        ASSISTANT_TO_ID = bot.get('assistant_id')
        pprint.pprint(bot)

print(f'\n\nYour assistant FROM id is: {ASSISTANT_FROM_ID}')
print(f'\n\nYour assistant TO id is: {ASSISTANT_TO_ID}')

{'assistant_environments': [{'environment': 'draft',
                             'environment_id': '572332fa-74b2-4e42-9c4d-93c891106824',
                             'name': 'draft'},
                            {'environment': 'live',
                             'environment_id': '646fce51-a4ec-4bc8-8928-ef4eb222a59a',
                             'name': 'live'}],
 'assistant_id': '9dcbdb4b-f743-4a96-b20b-51f80f9ec4d7',
 'assistant_skills': [{'skill_id': '79fa487e-fc14-4f85-89b9-8f00c88f0ff7',
                       'type': 'dialog'},
                      {'skill_id': '5cbe6f41-8364-4f69-9e0a-1e7891cdde76',
                       'type': 'action'}],
 'description': 'Main Assistant for Ask Procurement',
 'language': 'en',
 'multi_lingual': {'enabled': False},
 'name': 'Ask_Proc_Main'}
{'assistant_environments': [{'environment': 'draft',
                             'environment_id': '136ba64a-3fd9-4edc-ba0f-788f14711d3c',
                             'name': 'draft'},
           

# Get the skills data from your assistant.

In [13]:
'''
A successful call to this method only initiates an asynchronous export. 
The exported JSON data is not available until processing completes.

After the initial request is submitted, you can poll the status of the operation by calling the same request again and 
checking the value of the status property. If an error occurs (indicated by a status value of Failed), 
the status_description property provides more information about the error, and the status_errors property 
contains an array of error messages that caused the failure.

When processing has completed, the request returns the exported JSON data. Remember that the usual rate limits apply.
'''

skills_from = assistant.export_skills(assistant_id=ASSISTANT_FROM_ID)

In [14]:
# here you get the json data 
assistand_from_data = skills_from.result
assistand_from_data

{'assistant_skills': [{'name': 'Ask_Proc_Main',
   'type': 'dialog',
   'status': 'Available',
   'language': 'en',
   'skill_id': '79fa487e-fc14-4f85-89b9-8f00c88f0ff7',
   'workspace': {'intents': [],
    'entities': [],
    'metadata': {'api_version': {'major_version': 'v2',
      'minor_version': '2021-06-14'}},
    'dialog_nodes': [],
    'counterexamples': [],
    'learning_opt_out': False},
   'description': 'created for assistant 9dcbdb4b-f743-4a96-b20b-51f80f9ec4d7',
   'dialog_settings': {}},
  {'name': 'Ask_Proc_Main-action',
   'type': 'action',
   'status': 'Available',
   'language': 'en',
   'skill_id': '5cbe6f41-8364-4f69-9e0a-1e7891cdde76',
   'workspace': {'actions': [{'steps': [{'step': 'step_987',
        'output': {'generic': []},
        'context': {'variables': [{'value': {'scalar': True},
           'skill_variable': 'child_flow_live'}]},
        'handlers': [],
        'resolver': {'type': 'continue'},
        'variable': 'step_987',
        'condition': {'and'

In [15]:
assistand_from_data.keys()

dict_keys(['assistant_skills', 'assistant_state'])

# Transform the assistant_skills data into text 

In [16]:
assistant_from_skills_txt = str(assistand_from_data['assistant_skills'])

In [17]:

file_name = BACKUP_FOLDER+'/'+ASSISTANT_FROM+'_DOWNLOAD_'+date_name+'.txt'
print(file_name)

with open(file=file_name, mode='w') as f:
    f.write(assistant_from_skills_txt)
    f.close()

assistant_backups/2024-02-20_11_45_25/Ask_Proc_Main_DOWNLOAD_2024-02-20_11_45_25.txt


# Process to replace strings in assistant 

With the data in string format you can now use the replace method to replace the catalog and hash ids 

In [18]:
for i in MODIFY:
    assistant_from_skills_txt = assistant_from_skills_txt.replace(i['id_from'], i['id_to'])

# After replacement 

Once the ids are replaced , now we can transform the text again to a list using eval function. 

In [19]:
# saving the file as upload 
upload_name = BACKUP_FOLDER+'/'+ASSISTANT_FROM+'_UPLOAD_'+date_name+'.txt'
print(upload_name)

with open(file=upload_name, mode='w') as f:
    f.write(assistant_from_skills_txt)
    f.close()

assistant_backups/2024-02-20_11_45_25/Ask_Proc_Main_UPLOAD_2024-02-20_11_45_25.txt


In [20]:
# then we can go back to list here.
assistant_from_skills_list = eval(assistant_from_skills_txt)

# Upload updated skills 

Now we can upload the skills data again in the server 

In [21]:
import_skills = assistant.import_skills(assistant_id=ASSISTANT_TO_ID, 
                                        include_audit=True, 
                                        assistant_skills=assistant_from_skills_list, 
                                        assistant_state = assistand_from_data['assistant_state'])

# Check the import state 

In [24]:
assistant.import_skills_status(assistant_id=ASSISTANT_TO_ID).get_result()

{'status': 'Completed', 'assistant_id': 'b8072bb0-0377-4000-a1b5-1dcc9059ecd7'}